In [33]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from datetime import datetime

# =============================================================================
# 1. LOAD THE DATA
# =============================================================================
df = pd.read_csv('acndata_sessions.csv')

# =============================================================================
# 2. DATA CLEANING AND FILTERING
# =============================================================================
df.dropna(subset=['sessionID', 'connectionTime', 'disconnectTime', 'doneChargingTime', 'kWhDelivered'], inplace=True)

# Convert to datetime
for col in ['connectionTime', 'disconnectTime', 'doneChargingTime']:
    df[col] = pd.to_datetime(df[col], errors='coerce')

# Drop rows with failed datetime conversion
df.dropna(subset=['connectionTime', 'disconnectTime'], inplace=True)

# Calculate session duration in hours (using disconnectTime)
df['session_duration'] = (df['disconnectTime'] - df['connectionTime']).dt.total_seconds() / 3600.0

# Filter out non-positive and extreme durations
df = df[(df['session_duration'] > 0) & (df['session_duration'] < 12)]

# =============================================================================
# Normalize hour: shift such that 8:00 becomes 0 (the garage is in Los Angeles; times are in GMT)
# =============================================================================
df['hour'] = (df['connectionTime'].dt.hour - 8) % 24

# =============================================================================
# PLOTTING ARRIVAL RATE AND AVERAGE SESSION DURATION BY HOUR
# =============================================================================
# Compute arrival rate and average session duration
arrival_rate = df.groupby('hour').size()
avg_duration = df.groupby('hour')['session_duration'].mean()

# Convert to DataFrame for Plotly
metrics_df = pd.DataFrame({
    'hour': arrival_rate.index,
    'arrival_rate': arrival_rate.values,
    'avg_duration': avg_duration.values
})

# Plot arrival rate
fig1 = px.line(metrics_df, x='hour', y='arrival_rate', markers=True,
               labels={'hour': 'Hour of Day', 'arrival_rate': 'Number of Sessions'},
               title='Session Arrival Rate by Hour')
fig1.update_layout(xaxis=dict(tickmode='linear', tick0=0, dtick=1))

# Plot average session duration
fig2 = px.line(metrics_df, x='hour', y='avg_duration', markers=True,
               labels={'hour': 'Hour of Day', 'avg_duration': 'Average Duration (hours)'},
               title='Average Session Duration by Hour')
fig2.update_layout(xaxis=dict(tickmode='linear', tick0=0, dtick=1))

fig1.show()
fig2.show()

print("Arrival Rate by Hour:")
print(arrival_rate.sort_index())
print("\nAverage Session Duration by Hour (in hours):")
print(avg_duration.sort_index())

# =============================================================================
# COMPUTE CHARGING DURATION AND POWER OUTPUT
# =============================================================================
# Calculate charging duration in hours (time during which the car charged)
df['charging_duration'] = (df['doneChargingTime'] - df['connectionTime']).dt.total_seconds() / 3600.0

# Filter out non-positive charging durations
df = df[df['charging_duration'] > 0]

# Calculate average power output in kilowatts (kW)
df['power_output'] = df['kWhDelivered'] / df['charging_duration']

# =============================================================================
# COMPUTE SYSTEM DEMAND OVER TIME
# =============================================================================
# For each session, assume constant power output during charging.
# Create event records: at connectionTime, add the session's power_output; at doneChargingTime, subtract it.
df_start = df[['connectionTime', 'power_output']].copy()
df_start.rename(columns={'connectionTime': 'timestamp'}, inplace=True)

df_end = df[['doneChargingTime', 'power_output']].copy()
df_end.rename(columns={'doneChargingTime': 'timestamp'}, inplace=True)
df_end['power_output'] = -df_end['power_output']

# Combine start and end events, then sort by time
events_df = pd.concat([df_start, df_end]).sort_values('timestamp')

# Compute the cumulative sum to derive the instantaneous total power demand
events_df['total_power'] = events_df['power_output'].cumsum()

# Set the timestamp as index for resampling
events_df = events_df.set_index('timestamp')

# Resample to daily frequency to obtain the average system demand per day throughout the year
daily_avg = events_df['total_power'].resample('D').mean().reset_index()

# =============================================================================
# PLOTTING AVERAGE SYSTEM DEMAND OVER THE YEAR
# =============================================================================
fig_demand = px.line(daily_avg,
                     x='timestamp',
                     y='total_power',
                     labels={'timestamp': 'Date', 'total_power': 'Average System Demand (kW)'},
                     title='Average kW Demand of the Garage Throughout the Year')
fig_demand.show()


Arrival Rate by Hour:
hour
0       46
1       17
2       21
3      128
4       57
5      326
6      867
7     2062
8     1441
9      681
10     401
11     530
12     530
13     377
14     402
15     490
16     553
17     665
18     538
19     362
20     245
21     195
22      93
23      59
dtype: int64

Average Session Duration by Hour (in hours):
hour
0     4.112675
1     3.810082
2     3.359193
3     6.904954
4     3.524381
5     6.025608
6     7.284035
7     7.137486
8     6.580886
9     5.617128
10    4.296327
11    4.103747
12    3.649209
13    2.840964
14    2.404449
15    2.285677
16    2.339807
17    2.285556
18    2.880271
19    3.364014
20    3.724796
21    3.975192
22    4.283811
23    4.349379
Name: session_duration, dtype: float64


In [10]:
# Print out the available columns for verification
print("Columns in CSV:", df.columns.tolist())

# Count and print the number of unique stations, spaces, and sites
if 'stationID' in df.columns:
    num_stations = df['stationID'].nunique()
    print("Number of unique stations:", num_stations)
else:
    print("Column 'stationID' not found in data.")

if 'spaceID' in df.columns:
    num_spaces = df['spaceID'].nunique()
    print("Number of unique spaces:", num_spaces)
else:
    print("Column 'space' not found in data.")

if 'siteID' in df.columns:
    num_sites = df['siteID'].nunique()
    print("Number of unique sites:", num_sites)
else:
    print("Column 'site' not found in data.")

Columns in CSV: ['_id', 'clusterID', 'connectionTime', 'disconnectTime', 'doneChargingTime', 'kWhDelivered', 'sessionID', 'siteID', 'spaceID', 'stationID', 'timezone', 'userID', 'userInputs', 'session_duration', 'hour']
Number of unique stations: 54
Number of unique spaces: 54
Number of unique sites: 1


In [11]:
!pip install plotly

In [12]:
import plotly.express as px

station_counts = df['stationID'].value_counts().reset_index()
station_counts.columns = ['stationID', 'Usage']

space_counts = df['spaceID'].value_counts().reset_index()
space_counts.columns = ['spaceID', 'Usage']

site_counts = df['siteID'].value_counts().reset_index()
site_counts.columns = ['siteID', 'Usage']

# Create an interactive bar chart for station usage
fig_station = px.bar(
    station_counts,
    x='stationID',
    y='Usage',
    title='Usage by Station',
    labels={'stationID': 'Station ID', 'Usage': 'Number of Sessions'}
)
fig_station.show()

# Create an interactive bar chart for space usage
fig_space = px.bar(
    space_counts,
    x='spaceID',
    y='Usage',
    title='Usage by Space',
    labels={'space': 'Space', 'Usage': 'Number of Sessions'}
)
fig_space.show()

# Create an interactive bar chart for site usage
fig_site = px.bar(
    site_counts,
    x='siteID',
    y='Usage',
    title='Usage by Site',
    labels={'site': 'Site', 'Usage': 'Number of Sessions'}
)
fig_site.show()